In [1]:
import yaml
import os
import glob
import json
from datetime import datetime
from bs4 import BeautifulSoup
import copy

prj = r"Z:\LWI\FIA Darlington\AmiteWatershed_2016Event_WithDarlingtonReservoir\AmiteWatershed_2016Event.prj"
prj_dir, prj_file_tail = os.path.split(prj)
prj_name = prj_file_tail.split(".")[0]

# coordinate_system
with open(f'{prj_dir}/{prj_name}.projection', "r") as f:
     coordinate_system = f.read()

with open(prj, "r") as f:
     lines = f.readlines()
lines = [s.strip('\n') for s in lines]
# lines

In [2]:

keyValueList = copy.deepcopy(lines)  

# get fia version, author, and date
for i,v in enumerate(lines):
        if 'Version=' in v:
                proj_fia_version = v.split('=')[-1]
        if 'Created=' in v:
                # Timestamp is in milliseconds. Divided by 1000 to get timestamp in seconds
                proj_created_date_unix = int(v.split('=')[-1])/1000
                # Convert to datetime format 
                proj_created_date = datetime.utcfromtimestamp(proj_created_date_unix).strftime("%Y-%m-%d")

        if 'Created By=' in v:
                proj_author = v.split('=')[-1]
        
# Find the first wanted header to start parsing Map and Manager blocks, get the line number, break the loop.
for i,v in enumerate(lines):
        if (v == 'MapBegin') or (v == 'ManagerBegin'):
                startLine_after_proj_info = i
                break

# Parse the wanted blocks of Manager and Map.
nestList = []
for i,v in enumerate(lines):
        if (v == 'ManagerEnd') or (v == 'MapEnd'):
                # If not the beginning of the file, skip a blank line (+1) for the start of the subList.
                if len(nestList) > 0:
                        nestList.append(lines[startLine_after_proj_info+1:i])
                else:
                        nestList.append(lines[startLine_after_proj_info:i])
                startLine_after_proj_info = i+1



In [11]:
# Break the nestList into 2 lists each for Maps and Managers by chcking the first value in each subList.
# Only appending the values in each subList from columns [1:4] for mapList and [1:5] for managerList. The others deemed unnecessary.
mapList = []
managerList = []
for subList in nestList:
    if subList[0] == 'MapBegin':
        mapList.append(subList[1:4])
    if subList[0] == 'ManagerBegin':
        managerList.append(subList[1:5])

In [12]:
mapList

[['Name=Amite_StructureInventory_NAD83_Albers.shp',
  'Description=',
  'Path=maps/Amite_StructureInventory_NAD83_Albers.shp',
  'Shown=true'],
 ['Name=AmiteParishes_NAD83_Albers.shp',
  'Description=',
  'Path=maps/AmiteParishes_NAD83_Albers.shp',
  'Shown=true'],
 ['Name=AmiteHUC8_NAD83_Albers.shp',
  'Description=',
  'Path=maps/AmiteHUC8_NAD83_Albers.shp',
  'Shown=true']]

In [13]:
# spatial extent - ManagerList - Class=hec2.fia.model.TerrainModelManager - File=watershed/terrain grids/amite_ws_10ft_dem.tgrid
# Find subList in ManagerList containing Class=hec2.fia.model.TerrainModelManager
managerList
# get all class types


[['Name=Program Order Manager',
  'Description=',
  'File=shared/programOrder.xml',
  'Class=hec2.model.ProgramOrderManager'],
 ['Name=Study',
  'Description=',
  'File=study/Study.dbf',
  'Class=hec2.watershed.model.StudyStudyManager'],
 ['Name=Computation Point',
  'Description=',
  'File=study/ComputationPoint.dbf',
  'Class=hec2.watershed.model.ComputationPointManager'],
 ['Name=Channel Mod',
  'Description=',
  'File=study/ChannelMod.dbf',
  'Class=hec2.watershed.model.ChannelModManager'],
 ['Name=Configuration',
  'Description=',
  'File=study/Condition.dbf',
  'Class=hec2.watershed.model.ConditionManager'],
 ['Name=Diversion',
  'Description=',
  'File=study/Diversion.dbf',
  'Class=hec2.watershed.model.DiversionManager'],
 ['Name=ImpactArea',
  'Description=',
  'File=study/ImpactArea.dbf',
  'Class=hec2.watershed.model.ImpactAreaManager'],
 ['Name=Levee',
  'Description=',
  'File=study/Levee.dbf',
  'Class=hec2.watershed.model.LeveeManager'],
 ['Name=OtherProject',
  'Descrip

In [26]:
print (managerList[0])

['Name=Program Order Manager', 'Description=', 'File=shared/programOrder.xml', 'Class=hec2.model.ProgramOrderManager']


In [28]:
manager_kv_list = []
for subList in managerList:
    kv = {
        # Name
        subList[0].split('=')[0] : subList[0].split('=')[-1],
        # Description
        subList[1].split('=')[0] : subList[1].split('=')[-1],
        # File
        subList[2].split('=')[0] : subList[2].split('=')[-1],
        # Class
        subList[3].split('=')[0] : subList[3].split('=')[-1].split('.')[-1],   
    }
    manager_kv_list.append(kv)
manager_kv_list

[{'Name': 'Program Order Manager',
  'Description': '',
  'File': 'shared/programOrder.xml',
  'Class': 'ProgramOrderManager'},
 {'Name': 'Study',
  'Description': '',
  'File': 'study/Study.dbf',
  'Class': 'StudyStudyManager'},
 {'Name': 'Computation Point',
  'Description': '',
  'File': 'study/ComputationPoint.dbf',
  'Class': 'ComputationPointManager'},
 {'Name': 'Channel Mod',
  'Description': '',
  'File': 'study/ChannelMod.dbf',
  'Class': 'ChannelModManager'},
 {'Name': 'Configuration',
  'Description': '',
  'File': 'study/Condition.dbf',
  'Class': 'ConditionManager'},
 {'Name': 'Diversion',
  'Description': '',
  'File': 'study/Diversion.dbf',
  'Class': 'DiversionManager'},
 {'Name': 'ImpactArea',
  'Description': '',
  'File': 'study/ImpactArea.dbf',
  'Class': 'ImpactAreaManager'},
 {'Name': 'Levee',
  'Description': '',
  'File': 'study/Levee.dbf',
  'Class': 'LeveeManager'},
 {'Name': 'OtherProject',
  'Description': '',
  'File': 'study/OtherProject.dbf',
  'Class': '

In [30]:
import pandas as pd
df = pd.DataFrame(manager_kv_list)
df

,Name,Description,File,Class
0,Program Order Manager,,shared/programOrder.xml,ProgramOrderManager
1,Study,,study/Study.dbf,StudyStudyManager
2,Computation Point,,study/ComputationPoint.dbf,ComputationPointManager
3,Channel Mod,,study/ChannelMod.dbf,ChannelModManager
4,Configuration,,study/Condition.dbf,ConditionManager
5,Diversion,,study/Diversion.dbf,DiversionManager
6,ImpactArea,,study/ImpactArea.dbf,ImpactAreaManager
7,Levee,,study/Levee.dbf,LeveeManager
8,OtherProject,,study/OtherProject.dbf,OtherProjectManager
9,Reservoir,,study/Reservoir.dbf,ReservoirManager
